In [1]:
from math import sqrt
import numpy as np
import pandas as pd

from scipy.stats import f,friedmanchisquare, rankdata
from __future__ import division

# Friedman Test

In [2]:
def X_F_sqr(k,N,R):
    return ((12*N)/(k*(k+1)))*(np.sum(R**2)-(k*(k+1)**2)/4)

def F_F(k,N,X_F):
    return ((N-1)*X_F)/(N*(k-1)-X_F)

def critical_value(k, N, a=0.05):
    d1 = k - 1
    d2 = (k-1)*(N-1)
    return f.isf(a, d1, d2)

def cd(k,N,q_a):
    return q_a * sqrt((k*(k+1))/(6*N))

In [3]:
scores = pd.read_csv('./scores.csv')
classifiers = list(set(scores.columns) - set(['Dataset']))
#scores_data = scores[classifiers].values
scores_data = scores[list(scores.columns)[1:]].values
scores

,Dataset,M_G,CEM_SG_Vo,CEM_SG_MeL,M_SG,M_GA,M_SGA,CEM_SGA_Vo,CEM_SGA_MeL
0,SemEval2013,85.34,87.78,87.87,86.36,87.54,86.26,86.26,86.97
1,SemEval2014,86.16,84.16,87.63,87.03,88.05,86.94,85.90,88.07
2,Vader,87.71,87.92,88.85,88.07,88.89,88.89,89.52,89.48
3,STS,83.43,83.52,84.56,84.73,85.27,85.26,87.08,85.59


In [4]:
# parameters
k = scores_data.shape[1]
N=scores_data.shape[0]
a = 0.1

ranks = np.zeros(scores_data.shape)
for i,scores_ in enumerate(scores_data):
    ranks[i] = len(scores_)+1 - rankdata(scores_)
    
R = np.average(ranks, axis=0)

X_F = X_F_sqr(k=k,N=N,R=R)
print('k:', k, ' '*5, 'N:', N, ' '*5, 'a:', a)
print('chi2: ', X_F)
print("Friedman's F: ", F_F(k=k,N=N,X_F=X_F))
print('F({},{})|{}: '.format(k-1,(k-1)*(N-1),a), critical_value(k=k,N=N, a=a))

k: 8       N: 4       a: 0.1
chi2:  13.4791666667
Friedman's F:  2.78479196557
F(7,21)|0.1:  2.02325229746


In [5]:
t = pd.DataFrame(columns=list(scores.columns)[1:], index=[0])
t.ix[0] = R
t

,M_G,CEM_SG_Vo,CEM_SG_MeL,M_SG,M_GA,M_SGA,CEM_SGA_Vo,CEM_SGA_MeL
0,7.5,6,3.75,5,2.875,4.75,3.875,2.25


## Bonferroni-Dunn test

In [6]:
q_a = 2.450 # alpha = 0.1
baseline = 'M_G'
crit_diff = cd(k,N,q_a)
print('CD =', crit_diff)

bon_dunn = (t['M_G'].ix[0] - t > crit_diff) 
bon_dunn.ix[1] = (t['M_G'].ix[0] - t).values[0]
bon_dunn

CD = 4.243524478543749


,M_G,CEM_SG_Vo,CEM_SG_MeL,M_SG,M_GA,M_SGA,CEM_SGA_Vo,CEM_SGA_MeL
0,0.0,0.0,0.00,0.0,1.000,0.00,0.000,1.00
1,0.0,1.5,3.75,2.5,4.625,2.75,3.625,5.25


In [7]:
scores[classifiers]

,CEM_SGA_MeL,M_G,M_SGA,M_GA,CEM_SG_Vo,CEM_SG_MeL,M_SG,CEM_SGA_Vo
0,86.97,85.34,86.26,87.54,87.78,87.87,86.36,86.26
1,88.07,86.16,86.94,88.05,84.16,87.63,87.03,85.90
2,89.48,87.71,88.89,88.89,87.92,88.85,88.07,89.52
3,85.59,83.43,85.26,85.27,83.52,84.56,84.73,87.08


In [8]:
friedmanchisquare(*scores[classifiers].values[:4])

(18.599999999999994, 0.00033072142736429084)